In [1]:
import sys
import json
#if argv['params']['general'].get('scenario_path_S3') contains immense s3 path (ex: uuid-123-456-789/)
params = {'general':{'periods':['am','pm','ip','op']}}
default = {'scenario':'orchestrator2','training_folder': '../..', 'params': params} # Default execution parameters
# here sceneario: demand is only use not on lambda.
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
print(argv)


{'scenario': 'orchestrator2', 'training_folder': '../..', 'params': {'general': {'periods': ['am', 'pm', 'ip', 'op']}}}


In [2]:
import os
import time
import geopandas as gpd
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
from sklearn.neighbors import NearestNeighbors
from numba import jit, njit
import numba as nb
from quetzal.model import stepmodel
from shapely.geometry import LineString
from quetzal.io.gtfs_reader.importer import get_epsg
from quetzal.io import excel
import numpy as np

In [3]:
periods = argv['params']['general']['periods']
on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
print('On Lambda : ', on_lambda)
training_folder = argv['training_folder']
input_folder = training_folder +r'/inputs/'
if on_lambda:
    bucket_name = 'quetzal-immense' #os.environ.get('BUCKET_NAME')
    output_folder = training_folder + '/outputs/'
    scenario_folder = training_folder + '/inputs/'
    scenario_folders = []
    input_folders = []
    for p in periods:
        folder = 's3://' + bucket_name + '/' + p + '/'
        scenario_folders.append(folder)
        input_folders.append(input_folder + p + '/')

else:
    output_folder = training_folder + '/scenarios/' + argv['scenario'] + '/outputs/'
    scenario_folder = training_folder + '/scenarios/' + argv['scenario'] + '/inputs/'
    scenario_folders = []
    input_folders = []
    for p in periods:
        folder = training_folder + '/scenarios/' + p + '/'
        scenario_folders.append(folder)
        input_folders.append(input_folder + p + '/')
    
print('input folder: ', input_folders)
print('output folder: ', output_folder)
print('scen folder : ', scenario_folders)

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

On Lambda :  False
input folder:  ['../../inputs/am/', '../../inputs/pm/', '../../inputs/ip/', '../../inputs/op/']
output folder:  ../../scenarios/orchestrator2/outputs/
scen folder :  ['../../scenarios/am/', '../../scenarios/pm/', '../../scenarios/ip/', '../../scenarios/op/']


In [4]:
period_dict={'am':'AM','pm':'PM','ip':'IP','op':'OP'}

# Logit Parameters
logit_params ={'time': -0.00055,
             'price': -1.0,
             'transfers': -0.25,
             'mode': 1.0,
             'pt_mode': 0.5,
             'pt_path': 0.1}

# reference

In [5]:
# run that locally and save resulting geojson in inputs (will be dockerize)
if not on_lambda:
    volumes = pd.read_csv(input_folder+'base/volumes/volumes.csv')

    volumes.index.name = 'index'
    volumes['origin'] = 'zone_' + volumes['origin'].astype(str)
    volumes['destination'] = 'zone_' + volumes['destination'].astype(str)

    for period, ref_folder, scen_folder in zip(periods,input_folders,scenario_folders):
        print(period)

        sm = stepmodel.read_zippedpickles(scen_folder + 'model/ref_los')

        #volumes
        v = volumes[volumes['time_period'] == period_dict[period]]
        v = v[v['vehicle_class'].isin(['RPAX','BPAX'])]
        v = v.groupby(['origin','destination','vehicle_class'])['volume'].sum().unstack().reset_index().fillna(0)
        v.index.name='index'
        sm.volumes = v

        #links preparation
        rename = lambda ls: ['rlink_'+ str(x).replace('rlink_','') for x in ls]
        sm.links['road_link_list'] = sm.links['road_link_list'].apply(rename)
        sm.links.index.name='index'


        #logit prep
        sm.preparation_logit(segments=['BPAX', 'RPAX'], **logit_params)

        sm.segments = ['BPAX', 'RPAX']
        sm.mode_utility.loc['rail','BPAX'] = -2
        sm.mode_utility.loc['bus','RPAX'] = -2

        #sm.analysis_pt_los(walk_on_road=True)
        #sm.analysis_pt_route_type(hierarchy=[ 'car','rail', 'subway', 'tram', 'bus', 'walk'])

        sm.los = sm.pt_los
        sm.los = sm.los.reset_index()

        sm.los['time'] = sm.los['gtime']
        sm.los['price'] = 0

        # Logit
        sm.analysis_mode_utility(how='sum')

        sm.step_logit(n_paths_max=10, workers=1, nchunks=10)

        sm.step_assignment(
                road=True, 
                boardings=True, 
                alightings=True, 
                transfers=True,
                segmented=False,
                boarding_links=False,
                alighting_links=False)

        #export
        filename = input_folder+'{p}/loaded_links.geojson'.format(p=period)
        sm.links.to_crs(4326).drop(columns=['road_link_list']).to_file(filename,driver='GeoJSON')

        #sm.road_links.columns = [str(col) for col in sm.road_links.columns]
        #filename = input_folder+'{p}/loaded_road_links.geojson'.format(p=period)
        #sm.road_links.to_crs(4326).to_file(filename,driver='GeoJSON')


am


100%|███████████████████████████████████████████| 10/10 [00:01<00:00,  7.04it/s]


to assign boardings on links pass boarding_links=True
pm


100%|███████████████████████████████████████████| 10/10 [00:01<00:00,  6.91it/s]


to assign boardings on links pass boarding_links=True
ip


100%|███████████████████████████████████████████| 10/10 [00:01<00:00,  7.77it/s]


to assign boardings on links pass boarding_links=True
op


100%|███████████████████████████████████████████| 10/10 [00:01<00:00,  8.65it/s]


to assign boardings on links pass boarding_links=True


# affectation

In [6]:
volumes = pd.read_csv(output_folder+'volumes.csv')

volumes.index.name = 'index'
volumes['origin'] = 'zone_' + volumes['origin'].astype(str)
volumes['destination'] = 'zone_' + volumes['destination'].astype(str)

In [7]:
for period, ref_folder, scen_folder in zip(periods,input_folders,scenario_folders):
    print(period)

    sm = stepmodel.read_zippedpickles(scen_folder + 'model/los')

    #volumes
    v = volumes[volumes['time_period'] == period_dict[period]]
    v = v[v['vehicle_class'].isin(['RPAX','BPAX'])]
    v = v.groupby(['origin','destination','vehicle_class'])['volume'].sum().unstack().reset_index().fillna(0)
    v.index.name='index'
    sm.volumes = v

    #links preparation
    rename = lambda ls: ['rlink_'+ str(x).replace('rlink_','') for x in ls]
    sm.links['road_link_list'] = sm.links['road_link_list'].apply(rename)
    sm.links.index.name='index'


    #logit prep
    sm.preparation_logit(segments=['BPAX', 'RPAX'], **logit_params)

    sm.segments = ['BPAX', 'RPAX']
    sm.mode_utility.loc['rail','BPAX'] = -2
    sm.mode_utility.loc['bus','RPAX'] = -2

    #sm.analysis_pt_los(walk_on_road=True)
    #sm.analysis_pt_route_type(hierarchy=[ 'car','rail', 'subway', 'tram', 'bus', 'walk'])

    sm.los = sm.pt_los
    sm.los = sm.los.reset_index()

    sm.los['time'] = sm.los['gtime']
    sm.los['price'] = 0

    # Logit
    sm.analysis_mode_utility(how='sum')

    sm.step_logit(n_paths_max=10, workers=1, nchunks=10)

    sm.step_assignment(
            road=True, 
            boardings=True, 
            alightings=True, 
            transfers=True,
            segmented=False,
            boarding_links=False,
            alighting_links=False)
    
    #add reference values
    
    ref_links = gpd.read_file(input_folder+'{p}/loaded_links.geojson'.format(p=period))
    ref_links = ref_links.set_index('index')
    
    sm.links['volume_ref'] = ref_links['volume']
    sm.links['volume_diff'] = sm.links['volume'] - sm.links['volume_ref']
    
    sm.links['time_congestion_diff'] = sm.links['time_congestion'] - sm.links['time_ref']

    #export
    filename = output_folder+'loaded_links_{p}.geojson'.format(p=period)
    sm.links.to_crs(4326).drop(columns=['road_link_list']).to_file(filename,driver='GeoJSON')

    sm.road_links.columns = [str(col) for col in sm.road_links.columns]
    filename = output_folder+'loaded_road_links_{p}.geojson'.format(p=period)
    sm.road_links.to_crs(4326).to_file(filename,driver='GeoJSON')
    

am


100%|███████████████████████████████████████████| 10/10 [00:01<00:00,  7.27it/s]


to assign boardings on links pass boarding_links=True
pm


100%|███████████████████████████████████████████| 10/10 [00:01<00:00,  6.85it/s]


to assign boardings on links pass boarding_links=True
ip


100%|███████████████████████████████████████████| 10/10 [00:01<00:00,  6.90it/s]


to assign boardings on links pass boarding_links=True
op


100%|███████████████████████████████████████████| 10/10 [00:01<00:00,  9.45it/s]


to assign boardings on links pass boarding_links=True
